### **Preparing the [QMUL-OpenLogo](https://hangsu0730.github.io/qmul-openlogo/) Dataset for yolov5**

This notebook should
- Download the QMUL-OpenLogo dataset from Google Drive
- Create `/datasets/openlogo` with
    - yolo labels (.txt) in `/labels`
    - images (.jpg) in `/images`
    - classes and split (.txt) in `/details`
- Delete leftover files from download

In [ ]:
import xml.etree.ElementTree as et
import os
import shutil
import math
import tarfile
import gdown

In [ ]:
# Download openlogo.tar
os.mkdir('datasets')
gdown.download(id='1p1BWofDJOKXqCtO0JPT5VyuIPOsuxOuj', output='datasets/openlogo.tar', quiet=False)

In [ ]:
# Extract openlogo.tar
tarfile.open('datasets/openlogo.tar').extractall('datasets')
os.remove('datasets/openlogo.tar')
os.rename('datasets/openlogo', 'datasets/openlogo-voc')

In [ ]:
# Download supervised imageset split
gdown.download(id='1KBiq8Xi6jBkGKwfP10mO54BgjYlpij_n', output='datasets/openlogo-voc/imageSets/supervised_imageset.tar', quiet=False)
tarfile.open('datasets/openlogo-voc/imageSets/supervised_imageset.tar').extractall('datasets/openlogo-voc/imageSets/')
os.remove('datasets/openlogo-voc/imageSets/supervised_imageset.tar')

In [ ]:
# Create directories 
os.mkdir('datasets/openlogo')

os.mkdir('datasets/openlogo/images')
os.mkdir('datasets/openlogo/labels')
os.mkdir('datasets/openlogo/details')

os.mkdir('datasets/openlogo/images/train')
os.mkdir('datasets/openlogo/images/val')
os.mkdir('datasets/openlogo/images/test')

os.mkdir('datasets/openlogo/labels/train')
os.mkdir('datasets/openlogo/labels/val')
os.mkdir('datasets/openlogo/labels/test')

### Converting VOC annotations to yolo labels

In [ ]:
# Create class name to id map
# Initalize dict to hold instances
class_names = []

# Iterate through each XMLs
for filename in os.listdir('datasets/openlogo-voc/annotations'):
    xml_file = os.path.join('datasets/openlogo-voc/annotations', filename)
    root = et.parse(xml_file).getroot()

    # Get class name and add if needed
    for object_elem in root.findall('object'):
        class_name = object_elem.find('name').text
        if class_name not in class_names:
            class_names.append(class_name)
            
# Save class list in .txt
class_names.sort()
print('\n'.join(class_names), file=open('datasets/openlogo/details/class-list.txt', 'w'))

# Map ids to classes
class_name_id_map = {}
for i in range(len(class_names)):
    class_name_id_map[class_names[i]] = i

# Save map in .txt
formatted_map = []
for class_name in class_names:
    formatted_map.append('{}: {}'.format(class_name_id_map[class_name], class_name))
formatted_map = '\n'.join(formatted_map)
print(formatted_map, file=open('datasets/openlogo/details/class-id-map.txt', 'w'))

In [ ]:
# Generate yolo labels from each XML
# Iterate through each XML
for filename in os.listdir('datasets/openlogo-voc/annotations'):
    xml_file = os.path.join('datasets/openlogo-voc/annotations', filename)
    root = et.parse(xml_file).getroot()

    # Initialize dict to hold annotation data
    voc_annotation = {}
    voc_annotation['bnboxes'] = []

    # Parse XML Tree
    voc_annotation['id'] = os.path.splitext(root.find('filename').text)[0]
    size_elem = root.find('size')
    voc_annotation['size'] = {}
    voc_annotation['size']['width'] = int(size_elem.find('width').text)
    voc_annotation['size']['height'] = int(size_elem.find('height').text)
    
    for object in root.findall('object'):
        bnbox = {}
        bnbox['class'] = object.find('name').text
        bnbox_elem = object.find('bndbox')
        bnbox['xmin'] = int(bnbox_elem.find('xmin').text)
        bnbox['ymin'] = int(bnbox_elem.find('ymin').text)
        bnbox['xmax'] = int(bnbox_elem.find('xmax').text)
        bnbox['ymax'] = int(bnbox_elem.find('ymax').text)
        voc_annotation['bnboxes'].append(bnbox)

    # Initialize list to hold yolo label
    yolo_label = []

    # Iterate through each annotation box
    for bnbox in voc_annotation['bnboxes']:

        # Normalize Pascal VOC info for yolo label
        image_width = voc_annotation['size']['width']
        image_height = voc_annotation['size']['height']

        class_id = class_name_id_map[bnbox['class']]

        bnbox_x_center = ((bnbox['xmin'] + bnbox['xmax']) / 2) / image_width
        bnbox_y_center = ((bnbox['ymin'] + bnbox['ymax']) / 2) / image_height
        bnbox_width = (bnbox['xmax'] - bnbox['xmin']) / image_width
        bnbox_height = (bnbox['ymax'] - bnbox['ymin']) / image_height

        # Save normalized info to list
        yolo_label.append('{} {:.6f} {:.6f} {:.6f} {:.6f}'.format(class_id, bnbox_x_center, bnbox_y_center, bnbox_width, bnbox_height))

    # Save formatted annotation to .txt
    save_filename = voc_annotation['id'] + '.txt'
    save_file = os.path.join('datasets/openlogo/labels', save_filename)
    print('\n'.join(yolo_label), file=open(save_file, 'w'))

### Splitting Dataset

In [ ]:
# Copy supervised imageset split
for filename in os.listdir('datasets/openlogo-voc/imagesets/supervised_imageset'):
    src_file = os.path.join('datasets/openlogo-voc/imagesets/supervised_imageset', filename)
    dst_file = os.path.join('datasets/openlogo/details', filename)
    shutil.copy(src_file, dst_file)

#Parse supervised imageset split
test_split = open('datasets/openlogo/details/test.txt', 'r').read().split('\n')
test_split.pop()

val_split = open('datasets/openlogo/details/val.txt', 'r').read().split('\n')
val_split.pop()

train_split = open('datasets/openlogo/details/train.txt', 'r').read().split('\n')
train_split.pop()

In [ ]:
# Move images and labels to corresponding folder
for id in train_split:
    txt_file = os.path.join('datasets/openlogo/labels', id + '.txt')
    shutil.move(txt_file, 'datasets/openlogo/labels/train')

    jpg_file = os.path.join('datasets/openlogo-voc/JPEGImages', id + '.jpg')
    shutil.move(jpg_file, 'datasets/openlogo/images/train')

for id in val_split:
    txt_file = os.path.join('datasets/openlogo/labels', id + '.txt')
    shutil.move(txt_file, 'datasets/openlogo/labels/val')

    jpg_file = os.path.join('datasets/openlogo-voc/JPEGImages', id + '.jpg')
    shutil.move(jpg_file, 'datasets/openlogo/images/val')

for id in test_split:
    txt_file = os.path.join('datasets/openlogo/labels', id + '.txt')
    shutil.move(txt_file, 'datasets/openlogo/labels/test')

    jpg_file = os.path.join('datasets/openlogo-voc/JPEGImages', id + '.jpg')
    shutil.move(jpg_file, 'datasets/openlogo/images/test')

### Creating .yaml for training

In [1]:
# Create openlogo.yaml
yaml = '''path: ../datasets/openlogo
train: images/train
val: images/val
test: images/test

names:'''

class_id_file = 'datasets/openlogo/details/class-id-map.txt'
class_ids = open(class_id_file, 'r').read().split('\n')
class_ids.pop()

for class_id in class_ids:
    yaml += '\n  {}'.format(class_id)

print(yaml, file=open('yolov5/data/openlogo.yaml', 'w'))

# TO TRAIN: python train.py --data openlogo.yaml --weights yolov5s.pt --img 640 --batch-size 32 --epochs 300 --name logo-det
# TO RESUME: python train.py --resume

### DESTRUCTIVE: Cleanup

In [ ]:
# Remove leftover openlogo files
shutil.rmtree('datasets/openlogo-voc')